In [ ]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = 'mypassword'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

# Tracking the Most Frequently Returned Products

## Description of the use case
This use case involves tracking which products are returned most frequently to identify potential issues with product quality or customer satisfaction. By monitoring return patterns, businesses can take corrective actions to improve their offerings.

## Redis Data Structures Used
- **Count-Min Sketch**: To track the frequency of product returns.
- **Hashes**: To store return information for each product.
- **Sorted Sets**: To rank products based on their return frequency.

In [ ]:

# Mock data for the use case
mock_data = {
    'return_1001': {'product_id': 'product_1', 'reason': 'defective'},
    'return_1002': {'product_id': 'product_2', 'reason': 'not as described'},
    'return_1003': {'product_id': 'product_1', 'reason': 'defective'},
    'return_1004': {'product_id': 'product_3', 'reason': 'changed mind'},
    'return_1005': {'product_id': 'product_2', 'reason': 'not as described'},
    'return_1006': {'product_id': 'product_1', 'reason': 'defective'},
}


sketch = r.cms().initbyprob('return_sketch', 0.001, 0.002) # https://redis.io/docs/latest/develop/data-types/probabilistic/count-min-sketch/

# Use Hashes to store return information
for return_id, return_info in mock_data.items():
    r.hset(return_id, mapping=return_info)

# Use Count-Min Sketch to track return frequency
for return_info in mock_data.values():
    product_id = return_info['product_id']
    r.cms().incrby('return_sketch', [product_id], [1])

unique_products = list(set([mock_data[x]['product_id'] for x in mock_data.keys()]))

for x in unique_products:
    return_fq = r.cms().query('return_sketch',x)[0]
    print(f"Return frequency for {x}: {return_fq}")
    # Update a sorted set 
    r.zadd('return_frequency', {x:return_fq})

print("Most returned product:")
print(f"{r.zrevrange('return_frequency',0,0,withscores=True)[0]}")